In [1]:
import nltk, re, pprint
from nltk import load_parser

In [2]:
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [3]:
cp = load_parser('grammars/book_grammars/sql0.fcfg', trace=3)
query = 'What cities are located in China'.split()

In [4]:
#for x in cp.parse(query):
#    print(x)

In [5]:
#for x in cp.parse(query):
#    print(x.draw())

In [6]:
trees = list(cp.parse(query))
answer = trees[0].label()['SEM']
#answer = [x for x in answer if x]
q = ' '.join(answer)
print(q)

|.W.c.a.l.i.C.|
Leaf Init Rule:
|[-] . . . . .| [0:1] 'What'
|. [-] . . . .| [1:2] 'cities'
|. . [-] . . .| [2:3] 'are'
|. . . [-] . .| [3:4] 'located'
|. . . . [-] .| [4:5] 'in'
|. . . . . [-]| [5:6] 'China'
Feature Bottom Up Predict Combine Rule:
|[-] . . . . .| [0:1] Det[SEM='SELECT'] -> 'What' *
Feature Bottom Up Predict Combine Rule:
|[-> . . . . .| [0:1] NP[SEM=(?det+?n)] -> Det[SEM=?det] * N[SEM=?n] {?det: 'SELECT'}
Feature Bottom Up Predict Combine Rule:
|. [-] . . . .| [1:2] N[SEM='City FROM city_table'] -> 'cities' *
Feature Single Edge Fundamental Rule:
|[---] . . . .| [0:2] NP[SEM=(SELECT, City FROM city_table)] -> Det[SEM='SELECT'] N[SEM='City FROM city_table'] *
Feature Bottom Up Predict Combine Rule:
|[---> . . . .| [0:2] S[SEM=(?np+WHERE+?vp)] -> NP[SEM=?np] * VP[SEM=?vp] {?np: (SELECT, City FROM city_table)}
Feature Bottom Up Predict Combine Rule:
|. . [-] . . .| [2:3] IV[SEM=''] -> 'are' *
Feature Bottom Up Predict Combine Rule:
|. . [-> . . .| [2:3] VP[SEM=(?v+?pp)] 

In [7]:
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for x in rows:
    print(x[0], end=' ')

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

We need to update SQL grammar to translate the below query:

In [8]:
query2 = 'What cities are in China and have populations above 1,000,000'.split()
query3 = 'What cities have populations above 1,000,000'.split()

In [9]:
# specify and display user grammar

grammar_user = 'sql_grammar.fcfg'
nltk.data.show_cfg(grammar_user)

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?np)] -> IV[SEM=?v] NP[SEM=?np]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?n + ?pp)] -> N[SEM=?n] PP[SEM=?pp]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'
IV[SEM=''] -> 'have'
N[SEM='Population'] -> 'populations'
P[SEM='>'] -> 'above'
NP[SEM='1000'] -> '1,000,000'


In [10]:
cp = load_parser(grammar_user, trace=3)

In [11]:
trees = list(cp.parse(query3))
answer = trees[0].label()['SEM']
#answer = [x for x in answer if x]
q = ' '.join(answer)
#print(q)

|.W.c.h.p.a.1.|
Leaf Init Rule:
|[-] . . . . .| [0:1] 'What'
|. [-] . . . .| [1:2] 'cities'
|. . [-] . . .| [2:3] 'have'
|. . . [-] . .| [3:4] 'populations'
|. . . . [-] .| [4:5] 'above'
|. . . . . [-]| [5:6] '1,000,000'
Feature Bottom Up Predict Combine Rule:
|[-] . . . . .| [0:1] Det[SEM='SELECT'] -> 'What' *
Feature Bottom Up Predict Combine Rule:
|[-> . . . . .| [0:1] NP[SEM=(?det+?n)] -> Det[SEM=?det] * N[SEM=?n] {?det: 'SELECT'}
Feature Bottom Up Predict Combine Rule:
|. [-] . . . .| [1:2] N[SEM='City FROM city_table'] -> 'cities' *
Feature Bottom Up Predict Combine Rule:
|. [-> . . . .| [1:2] NP[SEM=(?n+?pp)] -> N[SEM=?n] * PP[SEM=?pp] {?n: 'City FROM city_table'}
Feature Single Edge Fundamental Rule:
|[---] . . . .| [0:2] NP[SEM=(SELECT, City FROM city_table)] -> Det[SEM='SELECT'] N[SEM='City FROM city_table'] *
Feature Bottom Up Predict Combine Rule:
|[---> . . . .| [0:2] S[SEM=(?np+WHERE+?vp)] -> NP[SEM=?np] * VP[SEM=?vp] {?np: (SELECT, City FROM city_table)}
Feature Bottom U

In [12]:
rows = chat80.sql_query('corpora/city_database/city.db', q)
for x in rows:
    print(x[0], end=' ')

athens bangkok barcelona berlin birmingham bombay bucharest budapest buenos_aires cairo calcutta canton chicago chungking delhi detroit glasgow hamburg hongkong_city hyderabad istanbul karachi kyoto leningrad london los_angeles madras madrid manila melbourne mexico_city milan montreal moscow mukden nagoya nanking naples new_york osaka paris peking philadelphia rio_de_janeiro rome santiago sao_paulo seoul shanghai singapore_city sydney tehran tientsin tokyo vienna yokohama 

In [13]:
q

'SELECT City FROM city_table WHERE  Population > 1000'

We now need to accommodate the second clause 